In [1]:
#q6
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Dataset
sentences = [
    "The sky is blue",
    "The sun is bright",
    "The grass is green",
    "The night is dark",
    "The stars are shining"
]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

max_len = max(len(seq) for seq in sequences)
padded = pad_sequences(sequences, maxlen=max_len, padding='post')

vocab_size = len(tokenizer.word_index) + 1

# Simple VAE
input_dim = max_len
latent_dim = 2

inputs = Input(shape=(input_dim,))
h = Dense(16, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

decoder_h = Dense(16, activation='relu')
decoder_output = Dense(input_dim, activation='sigmoid')

h_decoded = decoder_h(z)
outputs = decoder_output(h_decoded)

vae = Model(inputs, outputs)
vae.compile(optimizer='adam', loss='mse')

vae.fit(padded, padded, epochs=200, verbose=0)

reconstructed = vae.predict(padded)
print("Reconstructed Output:")
print(np.round(reconstructed))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Reconstructed Output:
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [6]:
#q7
# Install if not installed
!pip install -q transformers sentencepiece sacremoses

from transformers import MarianMTModel, MarianTokenizer

# Define input text FIRST
text = "Artificial Intelligence is transforming the world."

# Translation function
def translate(text, model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**tokens)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Translate
print("Original:", text)
print("French:", translate(text, "Helsinki-NLP/opus-mt-en-fr"))
print("German:", translate(text, "Helsinki-NLP/opus-mt-en-de"))

Original: Artificial Intelligence is transforming the world.


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Loading weights:   0%|          | 0/258 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

German: Künstliche Intelligenz verwandelt die Welt.


In [7]:
#q8
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim*2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.shape[0]
        hidden = hidden.repeat(seq_len, 1, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        return torch.softmax(attention, dim=0)

In [8]:
#q9
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

prompt = "Roses are red, violets are blue,"
poem = generator(prompt, max_length=40, num_return_sequences=1)

print(poem[0]['generated_text'])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'num_return_sequences', 'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Roses are red, violets are blue, and the heart is red.

The blood vessels on the outside of the heart are thicker than on the inside.

The heart's airways (myocardial and paracrine chambers) are smaller and have smaller, more flexible walls.

The veins are thinner than on the outside, and have smaller, more flexible walls.

The heart's blood vessels are much thinner than on the outside, and have thinner, less flexible walls.

The heart's arteries are larger and have smaller, more flexible walls.

The heart's blood vessels are larger and have larger, more flexible walls.

The heart's blood vessels are smaller and have smaller, more flexible walls.

The heart's blood vessels are larger and have larger, more flexible walls.

The heart's blood vessels are larger and have larger, more flexible walls.

The heart's blood vessels are smaller and have smaller, more flexible walls.

The heart's blood vessels are smaller and have smaller, more flexible walls.

The heart's blood vessels are smalle

In [10]:
# =========================================
# Q10 - Creative Writing Assistant

# =========================================

# Install transformers (safe version)
!pip install -q transformers sentencepiece

from transformers import pipeline

# Use text-generation instead of text2text-generation
generator = pipeline(
    task="text-generation",
    model="google/flan-t5-base"
)

# -----------------------------
# USER INPUT
# -----------------------------
genre = "Fantasy"
theme = "A hidden kingdom beneath the ocean"

# -----------------------------
# STORY PLOT PROMPT
# -----------------------------
plot_prompt = f"""
Write a detailed creative story plot.

Genre: {genre}
Theme: {theme}

Include:
- Main conflict
- Setting
- Unique twist
- Ending hook
"""

# Generate story plot
plot_output = generator(
    plot_prompt,
    max_new_tokens=200,
    temperature=0.9,
    do_sample=True
)

# -----------------------------
# CHARACTER PROMPT
# -----------------------------
character_prompt = f"""
Create a main character for a {genre} novel.

Include:
- Name
- Age
- Personality
- Background
- Strength
- Weakness
"""

# Generate character
character_output = generator(
    character_prompt,
    max_new_tokens=200,
    temperature=0.9,
    do_sample=True
)

# -----------------------------
# DISPLAY RESULTS
# -----------------------------
print("\n==============================")
print("  CREATIVE WRITING ASSISTANT  ")
print("==============================\n")

print("🎯 GENERATED STORY PLOT:\n")
print(plot_output[0]["generated_text"])

print("\n---------------------------------\n")

print("👤 GENERATED CHARACTER DESCRIPTION:\n")
print(character_output[0]["generated_text"])

print("\n==============================")
print("Generation Complete ✅")
print("==============================")

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausa


  CREATIVE WRITING ASSISTANT  

🎯 GENERATED STORY PLOT:


Write a detailed creative story plot.

Genre: Fantasy
Theme: A hidden kingdom beneath the ocean

Include:
- Main conflict
- Setting
- Unique twist
- Ending hook
oard Sunderbrook Reef off Oregon near Panama. Surborn, by his second childhood father Jack Dolph Whitemoor with little education available is forced the expedition starts its journey that would only find rich potential into world economy due on all points; at such great incipense height level to begin World Empire but little before falling victim

---------------------------------

👤 GENERATED CHARACTER DESCRIPTION:


Create a main character for a Fantasy novel.

Include:
- Name
- Age
- Personality
- Background
- Strength
- Weakness
er by raising two children while there lives stay dead so noone, or as alloran, comes through an act to detestes that have now returned which now has happened so to those toyin. These three youngsters seem unaware they had just done away dur